# 네이버 지식인 크롤링

## 오늘 날짜만 검색

In [ ]:
KEYWORD = '대선'
TODAY = datetime.today().strftime("%Y.%m.%d.")
print(TODAY)

In [ ]:
driver = webdriver.Chrome('./chromedriver')
driver.maximize_window()
base_url = "https://www.naver.com/"
driver.get(base_url)
time.sleep(1)

# 지식iN 으로 이동
nav = driver.find_element_by_id('NM_FAVORITE')
lis = nav.find_element_by_tag_name('ul').find_elements_by_tag_name('li')
lis[3].click()

# 키워드 검색 후 최신순 정렬
search_area = driver.find_element_by_class_name('search_area')
search_area.find_element_by_tag_name('input').send_keys(KEYWORD)    # 키워드 입력
driver.find_element_by_class_name('search_btn').click()     # 검색
driver.find_element_by_class_name('option_sort').find_elements_by_tag_name('li')[1].click()     # '최신순' 클릭
time.sleep(1)

# 기간 검색: TODAY
driver.find_element_by_id('au_date_select').click()
time.sleep(1)
from_date = driver.find_element_by_id('sel_from_date')
to_date = driver.find_element_by_id('sel_to_date')
# 시작날짜, 완료날짜 입력
actions = ActionChains(driver).double_click(from_date).send_keys(TODAY).double_click(to_date).send_keys(TODAY)
actions.perform()
driver.find_element_by_xpath('/html/body/div[3]/div/ul/li[4]/input[3]').click()

# 전체 페이지수 수집
while True:
    pagination = driver.find_element_by_class_name('s_paging')
    pages = pagination.find_elements_by_tag_name('a')
    tmp = []
    for page in pages:
        tmp.append(page.text)
    if '다음페이지' in tmp:
        pages[-1].click()
        time.sleep(2)
    else:
        NUM_OF_PAGE = int(pagination.text.split()[-1])
        break
print("전체 페이지 수: ", NUM_OF_PAGE)
driver.close()

In [ ]:
driver = webdriver.Chrome('./chromedriver')
driver.maximize_window()
ans_url_list = []; total_ans = 0

# 질문들의 url 수집
for page in range(NUM_OF_PAGE):
    kin_url = f'https://kin.naver.com/search/list.nhn?sort=date&query={quote(KEYWORD)}'
    period_url = f'&period={TODAY}%7C{TODAY}&section=kin'
    page_url = f'&page={page+1}'
    url = kin_url + period_url + page_url
    driver.get(url)
    time.sleep(1)
    #print(f"{page+1} 페이지")

    ul = driver.find_element_by_xpath('//*[@id="s_content"]/div[3]/ul')
    lis = ul.find_elements_by_tag_name('li')
    for li in lis:
        a_tag = li.find_element_by_tag_name('a')
        ans_url_list.append(a_tag.get_attribute('href'))

print(f'총 {len(ans_url_list)}개의 질문')

# 각 질문의 답변 개수
for i, ans_url in enumerate(ans_url_list):
    driver.get(ans_url)
    time.sleep(0.5)
    ans_cnt = int(driver.find_element_by_class_name('_answerCount.num').text)
    total_ans += ans_cnt
    print(f'{i+1}번째 : {ans_cnt}')
print("----------------------------------")
print(f'Total 답변 개수: {total_ans}')

driver.close()

In [ ]:
driver = webdriver.Chrome('./chromedriver')
driver.maximize_window()
answer_text_list = []

for ans_url in tqdm(ans_url_list):
    driver.get(ans_url)
    time.sleep(1)
    more = driver.find_element_by_xpath('//*[@id="nextPageButton"]')
    disp = more.get_attribute('style')      # 더보기 할 답변이 남아있지 않은 경우 "style:None" 이 존재한다
    try:
        # 더보기 할 답변이 남지 않을 때까지 while문을 돌린다
        while disp:     
            driver.find_element_by_id('nextPageButton').click()
            time.sleep(1)
    except:
        time.sleep(1)
    answers = driver.find_elements_by_class_name('answer-content__item._contentWrap._answer')
    for ans in answers:
        ps = ans.find_elements_by_tag_name('p')
        for p in ps:
            answer_text_list.append(p.text)
driver.close()

In [ ]:
len(answer_text_list)

In [ ]:
df = pd.DataFrame({
    'answer': answer_text_list
})
df.head()

In [ ]:
today = datetime.today().strftime("%y%m%d")
df.to_csv(f'./data/{today}_KIN_org.txt', sep='\t', index=False)

### 데이터 전처리

In [ ]:
df_test = pd.read_table(f'./data/{today}_KIN_org.txt')
df_test.head()

In [ ]:
df_test.shape

In [ ]:
# 공백 여부 확인
drop_idx = df_test[df_test.answer == ""].index
print(len(drop_idx), drop_idx)

In [ ]:
# null값 여부 확인
df_test.isnull().sum()

In [ ]:
# null값을 가지고 있는 행 제거
df_test.dropna(axis=0, inplace=True)
df_test.shape

In [ ]:
df_test.head()

In [ ]:
df_test.to_csv(f'./result/{today}_KIN_res.txt.txt', sep='\t', index=False)